# Error Case Analysis - Interactive Viewer

This notebook provides an interactive interface to analyze error cases from fingerprint identification experiments.

**Features:**
- Load results from a folder
- Navigate through error cases
- Visualize minutiae matching between query and impostor/genuine pairs
- Color-coded minutiae pairs with matching scores

## 0. Configuration

In [ ]:
# ========== CONFIGURATION ==========

# Results folder to load
RESULTS_FOLDER = "dmd_sd27+tse1k"  # Change this to your results folder

# Device for matching (if you want to recompute details)
DEVICE = 'cuda:4'

print(f"Configuration loaded!")
print(f"  Results folder: {RESULTS_FOLDER}")
print(f"  Device: {DEVICE}")

## 1. Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyArrowPatch
import numpy as np
import pickle as pkl
import os
from glob import glob
from tqdm.notebook import tqdm
import torch
from ipywidgets import interact, IntSlider, Dropdown, VBox, HBox, Button, Output, IntText
import ipywidgets as widgets

import dmd
import grids as gr
gr.visualization.plot_mnt

# Helper function
noext = lambda f: os.path.splitext(f)[0]

print("Imports complete!")

## 2. Load Results and Templates

In [ ]:
# Load identification results
identification_results_path = os.path.join(RESULTS_FOLDER, "identification_results.pkl")

print(f"Loading results from: {identification_results_path}")
with open(identification_results_path, "rb") as f:
    results = pkl.load(f)

scores_matrix = results["scores_matrix"]
target_matrix = results["target_matrix"]
query_ids = results["query_ids"]
gallery_ids = results["gallery_ids"]
metadata = results["metadata"]

print(f"✅ Results loaded!")
print(f"   Queries: {metadata['num_queries']}")
print(f"   Gallery (SD258): {metadata['num_gallery_sd258']}")
print(f"   Gallery (TS1K): {metadata['num_gallery_ts1k']}")
print(f"   Total gallery: {metadata['num_gallery_total']}")

In [ ]:
# Load datasets
print("Loading SD258 dataset...")
ds_sd258 = gr.datasets.SD258
ds_sd258.load(layers=["orig"])

queries = ds_sd258[{"sid": 0}]
gallery_sd258 = ds_sd258[{"sid": 1}]

print("Loading TS1K dataset...")
ds_ts1k = gr.datasets.TS1K
if metadata['num_gallery_ts1k'] > 0:
    ds_ts1k.load(layers=["orig"])
    gallery_ts1k_full = ds_ts1k[{"sid": 0}]
else:
    gallery_ts1k_full = []

print(f"✅ Datasets loaded!")
print(f"   Queries: {len(queries)}")
print(f"   Gallery SD258: {len(gallery_sd258)}")
print(f"   Gallery TS1K available: {len(gallery_ts1k_full)}")

In [ ]:
# Load templates from disk
print("Loading templates from disk...\n")

def move_template_to_device(template, device):
    """Move all tensors in a template to the specified device."""
    template_moved = {}
    for key, value in template.items():
        if isinstance(value, torch.Tensor):
            template_moved[key] = value.to(device)
        else:
            template_moved[key] = value
    return template_moved

# Query templates
query_folder = os.path.join(RESULTS_FOLDER, "query")
query_files = sorted(glob(os.path.join(query_folder, "*.pkl")))
query_templates = []
for f in tqdm(query_files, desc="Loading queries"):
    with open(f, "rb") as pkl_file:
        template = pkl.load(pkl_file)
        template = move_template_to_device(template, DEVICE)
        query_templates.append(template)

# SD258 gallery templates
gallery_sd258_folder = os.path.join(RESULTS_FOLDER, "gallery_sd258")
gallery_sd258_files = sorted(glob(os.path.join(gallery_sd258_folder, "*.pkl")))
gallery_sd258_templates = []
for f in tqdm(gallery_sd258_files, desc="Loading SD258 gallery"):
    with open(f, "rb") as pkl_file:
        template = pkl.load(pkl_file)
        template = move_template_to_device(template, DEVICE)
        gallery_sd258_templates.append(template)

# TS1K gallery templates (only what we used)
gallery_ts1k_folder = os.path.join(RESULTS_FOLDER, "gallery_ts1k")
gallery_ts1k_files = sorted(glob(os.path.join(gallery_ts1k_folder, "*.pkl")))
gallery_ts1k_files = gallery_ts1k_files[:metadata['num_gallery_ts1k'] if metadata['num_gallery_ts1k'] > 0 else None]
gallery_ts1k_templates = []
for f in tqdm(gallery_ts1k_files, desc="Loading TS1K gallery"):
    with open(f, "rb") as pkl_file:
        template = pkl.load(pkl_file)
        template = move_template_to_device(template, DEVICE)
        gallery_ts1k_templates.append(template)

# Combine gallery
gallery_templates = gallery_sd258_templates + gallery_ts1k_templates
gallery_ts1k = [gallery_ts1k_full[i] for i in range(len(gallery_ts1k_templates))]

print(f"\n✅ Templates loaded!")
print(f"   Query templates: {len(query_templates)}")
print(f"   Gallery templates: {len(gallery_templates)}")

## 3. Find Error Cases

In [ ]:
# Find error cases
sorted_indices = np.argsort(scores_matrix, axis=1)[:, ::-1]
error_cases = []

for q_idx in range(len(query_templates)):
    rank1_idx = sorted_indices[q_idx, 0]
    
    if not target_matrix[q_idx, rank1_idx]:
        # Find correct match
        correct_matches = np.where(target_matrix[q_idx])[0]
        if len(correct_matches) > 0:
            correct_idx = correct_matches[0]
            correct_rank = np.where(sorted_indices[q_idx] == correct_idx)[0][0] + 1
            
            error_cases.append({
                'query_idx': q_idx,
                'predicted_idx': rank1_idx,
                'correct_idx': correct_idx,
                'predicted_score': scores_matrix[q_idx, rank1_idx],
                'correct_score': scores_matrix[q_idx, correct_idx],
                'correct_rank': correct_rank
            })

print(f"🔴 Found {len(error_cases)} error cases")
print(f"   Error rate: {len(error_cases)/len(query_templates)*100:.2f}%")

## 4. Helper Functions

In [ ]:
# Create matcher
matcher = dmd.DmdMatcher()

def get_gallery_image(idx):
    """Get gallery image by index."""
    if idx < len(gallery_sd258):
        return gallery_sd258[idx]["orig"]
    else:
        ts1k_idx = idx - len(gallery_sd258)
        return gallery_ts1k[ts1k_idx]["orig"]

def get_gallery_id(idx):
    """Get gallery ID by index."""
    if idx < len(gallery_sd258):
        return f"SD258: {noext(gallery_sd258[idx].external_id)}"
    else:
        ts1k_idx = idx - len(gallery_sd258)
        return f"TS1K: {noext(gallery_ts1k[ts1k_idx].external_id)}"

def get_matching_details(query_tpl, gallery_tpl):
    """Get detailed matching information between two templates."""
    details = matcher.match(query_tpl, gallery_tpl, details=True)
    return details

print("✅ Helper functions defined!")

## 5. Visualization Function

In [ ]:
def draw_minutiae_with_matches(ax, img, mnt, matched_indices=None, matched_colors=None):
    """
    Draw minutiae on image with color-coded matches using grids visualization.
    
    Args:
        ax: matplotlib axis
        img: fingerprint image
        mnt: minutiae array [N, 3 or 4] with (x, y, angle, ...)
        matched_indices: list of indices of matched minutiae
        matched_colors: list of colors for matched minutiae
    """
    ax.imshow(img, cmap='gray')
    ax.axis('off')
    
    # Convert to numpy if needed
    if isinstance(mnt, torch.Tensor):
        mnt = mnt.cpu().numpy()
    
    # Handle shape [1, N, C] -> [N, C]
    if len(mnt.shape) == 3:
        mnt = mnt[0]
    
    # Filter out NaN minutiae
    valid_mask = ~np.isnan(mnt[:, 0])
    mnt_valid = mnt[valid_mask]
    
    if len(mnt_valid) == 0:
        return
    
    # Create color list for all minutiae
    has_matches = matched_indices is not None and len(matched_indices) > 0
    
    if has_matches:
        # Map matched indices to colors
        color_map = {idx: color for idx, color in zip(matched_indices, matched_colors)}
        
        # Create color list for all valid minutiae
        # Need to map from original indices to valid indices
        valid_indices = np.where(valid_mask)[0]
        colors = []
        for orig_idx in valid_indices:
            if orig_idx in color_map:
                colors.append(color_map[orig_idx])
            else:
                colors.append((0.8, 0.8, 0.8, 0.3))  # Light gray for unmatched
        
        # Draw matched minutiae with colors
        gr.visualization.plot_mnt(
            mnt_valid,
            ax=ax,
            degrees=True,  # DMD uses degrees
            custom_colors=colors,
            marker_style={'markersize': 8, 'markeredgewidth': 1.5},
            line_style={'scale': 15, 'linewidth': 1.5}
        )
    else:
        # Draw all minutiae in gray
        gr.visualization.plot_mnt(
            mnt_valid,
            ax=ax,
            degrees=True,  # DMD uses degrees
            marker_style={'markersize': 4, 'markeredgewidth': 0.8, 'markeredgecolor': 'lightgray'},
            line_style={'scale': 12, 'linewidth': 0.8, 'color': 'lightgray'}
        )


def draw_match_connections(ax, mnt1, mnt2, pairs, scores, offset_x=0):
    """
    Draw lines connecting matched minutiae pairs with score visualization.
    
    Args:
        ax: matplotlib axis
        mnt1: minutiae from query [N, 3+]
        mnt2: minutiae from gallery [N, 3+]
        pairs: matched pairs [M, 2]
        scores: scores for each pair [M]
        offset_x: horizontal offset for gallery image
    """
    # Convert to numpy
    if isinstance(mnt1, torch.Tensor):
        mnt1 = mnt1.cpu().numpy()
    if isinstance(mnt2, torch.Tensor):
        mnt2 = mnt2.cpu().numpy()
    
    # Handle shape [1, N, C] -> [N, C]
    if len(mnt1.shape) == 3:
        mnt1 = mnt1[0]
    if len(mnt2.shape) == 3:
        mnt2 = mnt2[0]
    
    # Normalize scores to [0, 1] for color mapping
    if len(scores) > 0:
        score_min, score_max = scores.min(), scores.max()
        if score_max > score_min:
            normalized_scores = (scores - score_min) / (score_max - score_min)
        else:
            normalized_scores = np.ones_like(scores)
    else:
        return
    
    # Color map: red (low score) -> yellow -> green (high score)
    cmap = plt.cm.RdYlGn
    
    # Draw connections
    for (i, j), score, norm_score in zip(pairs, scores, normalized_scores):
        i, j = int(i), int(j)
        
        # Get coordinates
        x1, y1 = mnt1[i, 0], mnt1[i, 1]
        x2, y2 = mnt2[j, 0] + offset_x, mnt2[j, 1]
        
        if np.isnan(x1) or np.isnan(y1) or np.isnan(x2) or np.isnan(y2):
            continue
        
        # Line color and width based on score
        color = cmap(norm_score)
        linewidth = 0.5 + 2.5 * norm_score  # Thicker lines for better scores
        
        # Draw connection line
        ax.plot([x1, x2], [y1, y2], color=color, linewidth=linewidth, alpha=0.6, zorder=1)
        
        # Optionally draw score text at midpoint
        if len(pairs) <= 20:  # Only show text if not too many pairs
            mid_x, mid_y = (x1 + x2) / 2, (y1 + y2) / 2
            ax.text(mid_x, mid_y, f'{score:.2f}', fontsize=7, color='white',
                   bbox=dict(boxstyle='round,pad=0.2', facecolor=color, alpha=0.7, edgecolor='none'),
                   ha='center', va='center', zorder=20)


def visualize_match_comparison(query_idx, impostor_rank=1, show_connections=True):
    """
    Visualize query vs impostor and query vs genuine with matched minutiae highlighted.
    
    Args:
        query_idx: Index of the query in error_cases
        impostor_rank: Rank of impostor to show (1 = top impostor)
        show_connections: If True, show a side-by-side view with connection lines
    """
    if query_idx >= len(error_cases):
        print(f"Error case {query_idx} not found. Max: {len(error_cases)-1}")
        return
    
    err = error_cases[query_idx]
    q_idx = err['query_idx']
    correct_idx = err['correct_idx']
    genuine_rank = err['correct_rank']
    
    # Get impostor at specified rank
    impostor_idx = sorted_indices[q_idx, impostor_rank - 1]
    impostor_score = scores_matrix[q_idx, impostor_idx]
    
    # Get templates
    query_tpl = query_templates[q_idx]
    impostor_tpl = gallery_templates[impostor_idx]
    genuine_tpl = gallery_templates[correct_idx]
    
    # Get images
    query_img = queries[q_idx]["orig"]
    impostor_img = get_gallery_image(impostor_idx)
    genuine_img = get_gallery_image(correct_idx)
    
    # Get matching details
    print("Computing matching details...")
    impostor_details = get_matching_details(query_tpl, impostor_tpl)
    genuine_details = get_matching_details(query_tpl, genuine_tpl)
    
    # Extract minutiae and convert to numpy
    query_mnt = query_tpl['mnt']
    if isinstance(query_mnt, torch.Tensor):
        query_mnt = query_mnt.cpu().numpy()
    if len(query_mnt.shape) == 3:
        query_mnt = query_mnt[0]
        
    impostor_mnt = impostor_tpl['mnt']
    if isinstance(impostor_mnt, torch.Tensor):
        impostor_mnt = impostor_mnt.cpu().numpy()
    if len(impostor_mnt.shape) == 3:
        impostor_mnt = impostor_mnt[0]
        
    genuine_mnt = genuine_tpl['mnt']
    if isinstance(genuine_mnt, torch.Tensor):
        genuine_mnt = genuine_mnt.cpu().numpy()
    if len(genuine_mnt.shape) == 3:
        genuine_mnt = genuine_mnt[0]
    
    # Extract pairs and metadata from details
    def extract_used_pairs(details):
        """Extract only the pairs that were used in the final score calculation."""
        pairs = details['pairs'] if 'pairs' in details else np.array([])
        sorted_idx = details.get('sorted_indices', np.arange(len(pairs)))
        n_pair = details.get('n_pair', len(pairs))
        relaxed_scores = details.get('relaxed_scores', details.get('scores'))
        
        # Convert to numpy
        if isinstance(pairs, torch.Tensor):
            pairs = pairs.cpu().numpy()
        if isinstance(sorted_idx, torch.Tensor):
            sorted_idx = sorted_idx.cpu().numpy()
        if isinstance(relaxed_scores, torch.Tensor):
            relaxed_scores = relaxed_scores.cpu().numpy()
        
        # Convert n_pair to int scalar
        if isinstance(n_pair, torch.Tensor):
            n_pair = int(n_pair.cpu().item())
        elif isinstance(n_pair, np.ndarray):
            n_pair = int(n_pair.item())
        else:
            n_pair = int(n_pair)
        
        # Handle shape [1, N, 2] -> [N, 2]
        if len(pairs.shape) == 3:
            pairs = pairs[0]
        if len(sorted_idx.shape) > 1:
            sorted_idx = sorted_idx[0]
        if len(relaxed_scores.shape) > 1:
            relaxed_scores = relaxed_scores[0]
        
        # Select only the top n_pair pairs that were used
        if len(sorted_idx) > 0 and len(pairs) > 0:
            used_indices = sorted_idx[:n_pair]
            used_pairs = pairs[used_indices]
            used_scores = relaxed_scores[used_indices]
            return used_pairs, used_scores, n_pair
        
        return pairs, relaxed_scores, n_pair
    
    impostor_pairs, impostor_scores, impostor_n_pair = extract_used_pairs(impostor_details)
    genuine_pairs, genuine_scores, genuine_n_pair = extract_used_pairs(genuine_details)
    
    # Generate distinct colors
    def generate_colors(n):
        """Generate n distinct colors."""
        if n == 0:
            return []
        cmap = plt.colormaps.get_cmap('hsv')
        return [cmap(i / n) for i in range(n)]
    
    impostor_colors = generate_colors(len(impostor_pairs))
    genuine_colors = generate_colors(len(genuine_pairs))
    
    if show_connections:
        # Create figure with connection view
        fig = plt.figure(figsize=(24, 12))
        
        # Top: Impostor with connections
        ax1 = plt.subplot(2, 2, 1)
        # Create side-by-side view
        h1, w1 = query_img.shape[:2]
        h2, w2 = impostor_img.shape[:2]
        max_h = max(h1, h2)
        combined_imp = np.ones((max_h, w1 + w2), dtype=query_img.dtype) * 255
        combined_imp[:h1, :w1] = query_img
        combined_imp[:h2, w1:] = impostor_img
        
        ax1.imshow(combined_imp, cmap='gray')
        ax1.axis('off')
        
        # Draw minutiae on both sides
        impostor_q_indices = impostor_pairs[:, 0].astype(int) if len(impostor_pairs) > 0 else []
        impostor_g_indices = impostor_pairs[:, 1].astype(int) if len(impostor_pairs) > 0 else []
        
        # Draw connections
        draw_match_connections(ax1, query_mnt, impostor_mnt, impostor_pairs, impostor_scores, offset_x=w1)
        
        # Draw minutiae on top
        for idx, color in zip(impostor_q_indices, impostor_colors):
            if idx < len(query_mnt) and not np.isnan(query_mnt[idx, 0]):
                x, y = query_mnt[idx, 0], query_mnt[idx, 1]
                ax1.plot(x, y, 'o', markersize=8, markerfacecolor=color, markeredgecolor='white', 
                        markeredgewidth=1.5, zorder=10)
        
        for idx, color in zip(impostor_g_indices, impostor_colors):
            if idx < len(impostor_mnt) and not np.isnan(impostor_mnt[idx, 0]):
                x, y = impostor_mnt[idx, 0] + w1, impostor_mnt[idx, 1]
                ax1.plot(x, y, 'o', markersize=8, markerfacecolor=color, markeredgecolor='white',
                        markeredgewidth=1.5, zorder=10)
        
        ax1.set_title(f'IMPOSTOR (Rank-{impostor_rank}) - Score: {impostor_score:.4f} - {len(impostor_pairs)} pairs',
                     fontsize=12, fontweight='bold', color='red')
        
        # Bottom: Genuine with connections
        ax2 = plt.subplot(2, 2, 3)
        h3, w3 = genuine_img.shape[:2]
        max_h2 = max(h1, h3)
        combined_gen = np.ones((max_h2, w1 + w3), dtype=query_img.dtype) * 255
        combined_gen[:h1, :w1] = query_img
        combined_gen[:h3, w1:] = genuine_img
        
        ax2.imshow(combined_gen, cmap='gray')
        ax2.axis('off')
        
        genuine_q_indices = genuine_pairs[:, 0].astype(int) if len(genuine_pairs) > 0 else []
        genuine_g_indices = genuine_pairs[:, 1].astype(int) if len(genuine_pairs) > 0 else []
        
        # Draw connections
        draw_match_connections(ax2, query_mnt, genuine_mnt, genuine_pairs, genuine_scores, offset_x=w1)
        
        # Draw minutiae on top
        for idx, color in zip(genuine_q_indices, genuine_colors):
            if idx < len(query_mnt) and not np.isnan(query_mnt[idx, 0]):
                x, y = query_mnt[idx, 0], query_mnt[idx, 1]
                ax2.plot(x, y, 'o', markersize=8, markerfacecolor=color, markeredgecolor='white',
                        markeredgewidth=1.5, zorder=10)
        
        for idx, color in zip(genuine_g_indices, genuine_colors):
            if idx < len(genuine_mnt) and not np.isnan(genuine_mnt[idx, 0]):
                x, y = genuine_mnt[idx, 0] + w1, genuine_mnt[idx, 1]
                ax2.plot(x, y, 'o', markersize=8, markerfacecolor=color, markeredgecolor='white',
                        markeredgewidth=1.5, zorder=10)
        
        ax2.set_title(f'GENUINE (Rank-{genuine_rank}) - Score: {err["correct_score"]:.4f} - {len(genuine_pairs)} pairs',
                     fontsize=12, fontweight='bold', color='green')
        
        # Add colorbar legend for scores
        ax3 = plt.subplot(2, 2, 2)
        ax3.axis('off')
        if len(impostor_scores) > 0:
            ax3.text(0.1, 0.9, f"Impostor Match Statistics:", fontsize=11, fontweight='bold', color='red')
            ax3.text(0.1, 0.8, f"n_pair: {impostor_n_pair}", fontsize=10)
            ax3.text(0.1, 0.7, f"Score range: [{impostor_scores.min():.3f}, {impostor_scores.max():.3f}]", fontsize=10)
            ax3.text(0.1, 0.6, f"Mean score: {impostor_scores.mean():.3f}", fontsize=10)
            ax3.text(0.1, 0.4, "Line color: Red=low, Yellow=mid, Green=high", fontsize=9, style='italic')
            ax3.text(0.1, 0.3, "Line thickness: proportional to score", fontsize=9, style='italic')
        
        ax4 = plt.subplot(2, 2, 4)
        ax4.axis('off')
        if len(genuine_scores) > 0:
            ax4.text(0.1, 0.9, f"Genuine Match Statistics:", fontsize=11, fontweight='bold', color='green')
            ax4.text(0.1, 0.8, f"n_pair: {genuine_n_pair}", fontsize=10)
            ax4.text(0.1, 0.7, f"Score range: [{genuine_scores.min():.3f}, {genuine_scores.max():.3f}]", fontsize=10)
            ax4.text(0.1, 0.6, f"Mean score: {genuine_scores.mean():.3f}", fontsize=10)
            ax4.text(0.1, 0.4, "Line color: Red=low, Yellow=mid, Green=high", fontsize=9, style='italic')
            ax4.text(0.1, 0.3, "Line thickness: proportional to score", fontsize=9, style='italic')
        
    else:
        # Original 2x2 grid view
        fig = plt.figure(figsize=(20, 10))
        
        # [Previous 2x2 grid code here - keep as fallback]
        ax1 = plt.subplot(2, 2, 1)
        impostor_q_indices = impostor_pairs[:, 0].astype(int) if len(impostor_pairs) > 0 else []
        draw_minutiae_with_matches(ax1, query_img, query_mnt, 
                                    matched_indices=impostor_q_indices,
                                    matched_colors=impostor_colors)
        ax1.set_title(f'Query #{q_idx}', fontsize=12, fontweight='bold')
        
        ax2 = plt.subplot(2, 2, 2)
        impostor_g_indices = impostor_pairs[:, 1].astype(int) if len(impostor_pairs) > 0 else []
        draw_minutiae_with_matches(ax2, impostor_img, impostor_mnt,
                                    matched_indices=impostor_g_indices,
                                    matched_colors=impostor_colors)
        ax2.set_title(f'Impostor (Rank-{impostor_rank})', fontsize=12, color='red')
        
        ax3 = plt.subplot(2, 2, 3)
        genuine_q_indices = genuine_pairs[:, 0].astype(int) if len(genuine_pairs) > 0 else []
        draw_minutiae_with_matches(ax3, query_img, query_mnt,
                                    matched_indices=genuine_q_indices,
                                    matched_colors=genuine_colors)
        ax3.set_title(f'Query #{q_idx}', fontsize=12, fontweight='bold')
        
        ax4 = plt.subplot(2, 2, 4)
        genuine_g_indices = genuine_pairs[:, 1].astype(int) if len(genuine_pairs) > 0 else []
        draw_minutiae_with_matches(ax4, genuine_img, genuine_mnt,
                                    matched_indices=genuine_g_indices,
                                    matched_colors=genuine_colors)
        ax4.set_title(f'Genuine (Rank-{genuine_rank})', fontsize=12, color='green')
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print("\n" + "="*80)
    print(f"ERROR CASE #{query_idx} - Query {q_idx}: {noext(queries[q_idx].external_id)}")
    print("="*80)
    
    print(f"\n📊 IMPOSTOR (Rank-{impostor_rank}): {get_gallery_id(impostor_idx)}")
    print(f"   Score: {impostor_score:.6f}")
    print(f"   Used pairs (n_pair): {impostor_n_pair}")
    if len(impostor_pairs) > 0:
        print(f"   Relaxed scores: min={impostor_scores.min():.4f}, max={impostor_scores.max():.4f}, mean={impostor_scores.mean():.4f}")
    
    print(f"\n✅ GENUINE (Rank-{genuine_rank}): {get_gallery_id(correct_idx)}")
    print(f"   Score: {err['correct_score']:.6f}")
    print(f"   Used pairs (n_pair): {genuine_n_pair}")
    if len(genuine_pairs) > 0:
        print(f"   Relaxed scores: min={genuine_scores.min():.4f}, max={genuine_scores.max():.4f}, mean={genuine_scores.mean():.4f}")
    
    print(f"\n⚠️  Score difference: {impostor_score - err['correct_score']:.6f}")
    print("="*80)

print("✅ Visualization functions defined!")

## 6. Interactive Viewer

In [ ]:
# Create interactive widget
error_case_slider = IntSlider(
    value=0,
    min=0,
    max=len(error_cases) - 1,
    step=1,
    description='Error Case:',
    continuous_update=False,
    style={'description_width': '120px'},
    layout=widgets.Layout(width='500px')
)

impostor_rank_slider = IntSlider(
    value=1,
    min=1,
    max=min(20, metadata['num_gallery_total']),
    step=1,
    description='Impostor Rank:',
    continuous_update=False,
    style={'description_width': '120px'},
    layout=widgets.Layout(width='500px')
)

# Interactive visualization
interact(visualize_match_comparison,   
         query_idx=error_case_slider,
         impostor_rank=impostor_rank_slider);

## 7. Quick Navigation

Use the cells below for quick access to specific cases:

In [ ]:
# Quick view: specific error case
ERROR_CASE_IDX = 0
IMPOSTOR_RANK = 1

visualize_match_comparison(ERROR_CASE_IDX, IMPOSTOR_RANK)

## 8. Export Detailed Match Information

In [ ]:
def export_match_details_to_file(error_case_idx, impostor_rank, output_path):
    """
    Export detailed matching information to a text file.
    """
    err = error_cases[error_case_idx]
    q_idx = err['query_idx']
    correct_idx = err['correct_idx']
    impostor_idx = sorted_indices[q_idx, impostor_rank - 1]
    
    query_tpl = query_templates[q_idx]
    impostor_tpl = gallery_templates[impostor_idx]
    genuine_tpl = gallery_templates[correct_idx]
    
    impostor_details = get_matching_details(query_tpl, impostor_tpl)
    genuine_details = get_matching_details(query_tpl, genuine_tpl)
    
    with open(output_path, 'w') as f:
        f.write(f"Error Case #{error_case_idx}\n")
        f.write(f"Query: {query_ids[q_idx]}\n")
        f.write(f"\nImpostor (Rank-{impostor_rank}): {gallery_ids[impostor_idx]}\n")
        f.write(f"  Score: {scores_matrix[q_idx, impostor_idx]:.6f}\n")
        f.write(f"  Pairs: {impostor_details}\n")
        f.write(f"\nGenuine (Rank-{err['correct_rank']}): {gallery_ids[correct_idx]}\n")
        f.write(f"  Score: {err['correct_score']:.6f}\n")
        f.write(f"  Pairs: {genuine_details}\n")
    
    print(f"✅ Exported to: {output_path}")

# Example usage:
# export_match_details_to_file(0, 1, "error_case_0_details.txt")